#### **Read the JSON file as a Dataframe**

In [35]:
df = spark.read.option("multiline", "true").json("Files/bing-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-news.json".
display(df)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 37, Finished, Available)

SynapseWidget(Synapse.DataFrame, ba254276-10a8-4fd8-b8a0-ca612a2ff190)

#### **Selecting just the value column from the dataframe - include all JSON data we need**

In [36]:
df = df.select("value")
display(df)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 38, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3b2a5ff3-6eb5-4162-8457-2ad9067fc5e1)

#### **Explode the "value" column from the single row structure into multiple row structure**

In [37]:
from pyspark.sql.functions import explode

df_exploded = df.select(explode(df["value"]).alias("json_object"))
display(df_exploded)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 39, Finished, Available)

SynapseWidget(Synapse.DataFrame, dd2f63e2-8396-49f1-8984-9547f409dab9)

#### **Converting the exploded JSON dataframe to a single JSON string list**

In [38]:
json_list = df_exploded.toJSON().collect()

# to see the json structure of all the news articles from the list
# print(json_list)

# to see the json structure of one news article from the list
print(json_list[0])

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 40, Finished, Available)

{"json_object":{"about":[{"name":"Hayao Miyazaki","readLink":"https://api.bing.microsoft.com/api/v7/entities/90949a72-116a-7b0a-8181-e3753c92c12b"},{"name":"Animation","readLink":"https://api.bing.microsoft.com/api/v7/entities/b91017e4-167f-21eb-8d1a-060c9b44cf80"},{"name":"Oscar","readLink":"https://api.bing.microsoft.com/api/v7/entities/a9114379-c69e-433b-bbe7-ba0e311624bb"}],"category":"Entertainment","datePublished":"2024-03-12T06:22:00.0000000Z","description":"Ghibli, the Japanese studio that just won its second Oscar for feature animation for “The Boy and The Heron,” hasn’t said yet what it plans next.","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=OVFT.GVI_fgPPk_CxVu90cES8GC&pid=News","height":393,"width":700}},"name":"Japanese animation studio founder Miyazaki isn’t ready to retire just yet, after latest Oscar win","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.AlMAEy7MoWNz0OI_xSWQiw&pid=news"}},"name":"As

#### **To work with information its really easy to convert JSON string list to a JSON dictionary. This is how to do it using json.loads()**

In [39]:
import json

article = json.loads(json_list[0])

print(article)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 41, Finished, Available)

{'json_object': {'about': [{'name': 'Hayao Miyazaki', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/90949a72-116a-7b0a-8181-e3753c92c12b'}, {'name': 'Animation', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/b91017e4-167f-21eb-8d1a-060c9b44cf80'}, {'name': 'Oscar', 'readLink': 'https://api.bing.microsoft.com/api/v7/entities/a9114379-c69e-433b-bbe7-ba0e311624bb'}], 'category': 'Entertainment', 'datePublished': '2024-03-12T06:22:00.0000000Z', 'description': 'Ghibli, the Japanese studio that just won its second Oscar for feature animation for “The Boy and The Heron,” hasn’t said yet what it plans next.', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.GVI_fgPPk_CxVu90cES8GC&pid=News', 'height': 393, 'width': 700}}, 'name': 'Japanese animation studio founder Miyazaki isn’t ready to retire just yet, after latest Oscar win', 'provider': [{'_type': 'Organization', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=ODF.AlMAEy7Mo

#### **To get information from certain elements ex: "description"**

In [40]:
print(article['json_object']['description'])

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 42, Finished, Available)

Ghibli, the Japanese studio that just won its second Oscar for feature animation for “The Boy and The Heron,” hasn’t said yet what it plans next.


#### **Lets get more information from the JSON Dictionary**

In [41]:
# name
# description
# url
# image
# provider
# datePublished

# Can use :"Online JSON Parser" to see the structure of the JSON string - copy & paste the full string to the online tool.

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 43, Finished, Available)

In [42]:
print(article['json_object']['name'])
print(article['json_object']['category'])
print(article['json_object']['description'])
print(article['json_object']['url'])
print(article['json_object']['image']["thumbnail"]["contentUrl"])
print(article['json_object']['provider'][0]['name'])
print(article['json_object']['datePublished'])

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 44, Finished, Available)

Japanese animation studio founder Miyazaki isn’t ready to retire just yet, after latest Oscar win
Entertainment
Ghibli, the Japanese studio that just won its second Oscar for feature animation for “The Boy and The Heron,” hasn’t said yet what it plans next.
https://apnews.com/article/miyazaki-ghibli-animation-oscar-japan-filmmaking-f572248c232017f7e9e9801f9a0e2c0b
https://www.bing.com/th?id=OVFT.GVI_fgPPk_CxVu90cES8GC&pid=News
Associated Press
2024-03-12T06:22:00.0000000Z


#### **Now, will loop through all the JSON dictionary lists and get the data**

In [43]:
# Initialise empty lists
name = []
category = []
description = []
url = []
image = []
provider = []
datePublished = []

#process each JSON object in the list
for json_str in json_list:
    try:
        # Parse the JSON string into a dictionary
        article = json.loads(json_str)

        # This is optional - There might be data inconsistencies from the data comes from Bing API, in that case if some articles missing some data, we can write this if condition to check
        if article["json_object"].get("name") and article["json_object"].get("image", {}).get("thumbnail", {}).get("contentUrl"):

            # Extract information from the dictionary
            name.append(article["json_object"]["name"])
            category.append(article["json_object"]["category"])
            description.append(article['json_object']['description'])
            url.append(article['json_object']['url'])
            image.append(article['json_object']['image']["thumbnail"]["contentUrl"])
            provider.append(article['json_object']['provider'][0]['name'])
            datePublished.append(article['json_object']['datePublished'])
    
    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 45, Finished, Available)

Error processing JSON object: 'category'
Error processing JSON object: 'category'


In [44]:
url

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 46, Finished, Available)

['https://apnews.com/article/miyazaki-ghibli-animation-oscar-japan-filmmaking-f572248c232017f7e9e9801f9a0e2c0b',
 'https://www.cnbc.com/2024/03/12/european-markets-live-updates-stocks-news-us-inflation-data.html',
 'https://www.msn.com/en-us/sports/nfl/2024-nfl-free-agency-updates-tracker-for-monday-buzz-notable-moves-as-deals-fly-in/ar-BB1jHgi4',
 'https://www.msn.com/en-us/sports/nfl/luke-combs-torches-panthers-after-teams-latest-reported-roster-move/ar-BB1jJ78N',
 'https://www.nytimes.com/2024/03/11/arts/music/morgan-wallen-billboard-record-country.html',
 'https://www.msn.com/en-us/news/world/the-latest-hunger-worsens-in-gaza-as-palestinians-mark-the-start-of-the-holy-month-of-ramadan/ar-BB1jHa9O',
 'https://www.msn.com/en-us/news/crime/sen-bob-menendez-and-wife-plead-not-guilty-to-latest-obstruction-of-justice-charges/ar-BB1jHxZC',
 'https://www.nytimes.com/2024/03/11/world/europe/kate-middleton-photo-princess-wales.html',
 'https://www.msn.com/en-us/news/crime/dog-kills-baby-boy-

#### **Combine all the lists together and create dataframe with a defined schema**

In [45]:
from pyspark.sql.types import StructType, StructField, StringType

# Combine the lists
data = list(zip(name,category,description,url,image,provider,datePublished))

# Define Schema
schema = StructType([
        StructField("name", StringType(), True),
        StructField("category", StringType(), True),
        StructField("description", StringType(), True),
        StructField("url", StringType(), True),
        StructField("image", StringType(), True),
        StructField("provider", StringType(), True),
        StructField("datePublished", StringType(), True)
])

# Create Dataframe
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 47, Finished, Available)

In [46]:
display(df_cleaned)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 48, Finished, Available)

SynapseWidget(Synapse.DataFrame, 095fb25d-a999-4f40-b15b-d97a22c61597)

#### **Transform "datepublished" column data from timestamp to date data type**

In [47]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MM-yyyy"))

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 49, Finished, Available)

In [48]:
display(df_cleaned_final)
# display(df_cleaned_final.limit(5))

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 50, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7de49396-1622-44f6-a987-c03f4289bf12)

#### **Writing the final dataframe to the lakehouse db in Delta format**

In [49]:
# df_cleaned_final.write.format("delta").saveAsTable("bing_lake_db.tbl_latest_news")



# Traditional ways of populating data table using modes - overwrite, append

# df_cleaned_final.write.format("delta").mode('overwrite').saveAsTable("bing_lake_db.tbl_latest_news")
# This method is not ideal when you have terabytes of data - this will be waste of time and resources (will lead to performance issues), also you are loosing old data by overwriting 

# df_cleaned_final.write.format("delta").mode('append').saveAsTable("bing_lake_db.tbl_latest_news")
# This method will leads to data duplication and unwantedly increasing the size of data

# So, most of the time, the best way is "INCREMENTAL LOAD"
# aka "SQL MERGE" in data warehousing
# In data warehousing we also called this Slowly changing diemention - Type 1 & Type 2
# Type 1 ignores of keeping history - always overwrites the current value with the new value, No history preserverd
# Type 2 inserts the new value in a new row and keep the old record as well - by using "Flag" column we can keep track if the record is new or old 

# in this project, will go with type 1 as we do not need to preserve history of data

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 51, Finished, Available)

#### **Handling the error "AnalysisException" when the table is already exist by when this code runs**
#### df_cleaned_final.write.format("delta").saveAsTable("bing_lake_db.tbl_latest_news")  

#### **Incremental Loading**

In [50]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_latest_news'
    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:
    print("Table Already Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_df_cleaned_final source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.name = target_table.name OR
                   source_view.category = target_table.category OR
                   source_view.description = target_table.description OR
                   source_view.url = target_table.url OR
                   source_view.image = target_table.image OR
                   source_view.provider = target_table.provider OR
                   source_view.datePublished = target_table.datePublished

                   THEN UPDATE SET *
                   
                   WHEN NOT MATCHED THEN INSERT *
                   
                   """)

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 52, Finished, Available)

In [51]:
%%sql

SELECT count(*) 
FROM bing_lake_db.tbl_latest_news

StatementMeta(, d6dc6350-75fb-4943-b9d8-18ae95275b98, 53, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>